### Library installation 

In [6]:
import requests
from requests.auth import HTTPBasicAuth
import json
import pandas as pd
import keyring
import datetime
import os

### Jira detailas and credentials

In [7]:
jira_domain = 'https://forgeholidays.atlassian.net/'
email = 'deepankar.srigyan@sykescottages.co.uk'
project_key = 'DAB ORDER BY created DESC'

#Credential needs to be stored in window credential before running this.
api_token = keyring.get_password('JIRA_API', 'deepankar.srigyan@sykescottages.co.uk')

### Date to prefix on file name

In [8]:
# Required to put datetime in the start of the file name
import datetime
from datetime import date

In [9]:
today = date.today()
date_prefix = today.strftime("%Y%m%d")  # Format the date as YYYYMMDD

### Scraping the data from Jira with the help of JIRA API
#### Doing pagination by the loop as JIRA has limit to extract 100 issues at once only.
#### File exported in Excel

In [10]:
import requests
from requests.auth import HTTPBasicAuth
import os
import pandas as pd

# Ensure the JIRA domain starts with 'https://'
if not jira_domain.startswith('https://'):
    jira_domain = 'https://' + jira_domain

# Create JIRA API URL
url = f"{jira_domain}/rest/api/3/search"

# Initialize variables for pagination
start_at = 0
max_results = 100  # Set to the maximum typically allowed by JIRA
total_issues_fetched = 0
all_issues = []

# Loop to fetch issues in pages
while True:
    query = {
        'jql': f'project={project_key}',
        'startAt': start_at * max_results,  # Adjusted for pagination
        'maxResults': max_results
    }

    try:
        response = requests.get(
            url,
            headers={'Content-Type': 'application/json'},
            params=query,
            auth=HTTPBasicAuth(email, api_token),
            verify=True  # Consider setting this to True once SSL certificates are configured
        )
        response.raise_for_status()
        response_data = response.json()
        issues = response_data['issues']
        all_issues.extend(issues)
        total_issues_fetched += len(issues)
        print(f'Fetched {len(issues)} issues, total fetched: {total_issues_fetched}')
        if len(issues) < max_results:
            break  # Exit the loop if the last page of issues is fetched
        else:
            start_at += 1  # Move to the next page
    except requests.exceptions.RequestException as e:
        print(f'An error occurred: {e}')
        break

# Convert to DataFrame
df_issues = pd.json_normalize(all_issues)

# Construct the file name with the date prefix
path = r"C:\Users\DeepankarSrigyan\OneDrive - Sykes Cottages Ltd\Documents - Data & Analytics\Insight team\1. Admin\4. Internal reporting\1. Jira"
file_name = os.path.join(path, f"{date_prefix}_jira_issues.csv")

# Save the DataFrame to a CSV file
df_issues.to_csv(file_name, index=False)

print(f'Total issues fetched: {total_issues_fetched}')
print(f'Issues have been saved to {file_name}')

Fetched 100 issues, total fetched: 100
Fetched 100 issues, total fetched: 200
Fetched 100 issues, total fetched: 300
Fetched 100 issues, total fetched: 400
Fetched 100 issues, total fetched: 500
Fetched 100 issues, total fetched: 600
Fetched 100 issues, total fetched: 700
Fetched 100 issues, total fetched: 800
Fetched 100 issues, total fetched: 900
Fetched 100 issues, total fetched: 1000
Fetched 100 issues, total fetched: 1100
Fetched 100 issues, total fetched: 1200
Fetched 100 issues, total fetched: 1300
Fetched 100 issues, total fetched: 1400
Fetched 100 issues, total fetched: 1500
Fetched 59 issues, total fetched: 1559
Total issues fetched: 1559
Issues have been saved to C:\Users\DeepankarSrigyan\OneDrive - Sykes Cottages Ltd\Documents - Data & Analytics\Insight team\1. Admin\4. Internal reporting\1. Jira\20240510_jira_issues.csv


#### End